In [1]:
import pandas as pd
import numpy as np

### Get house candidates from wmp 

In [2]:
h = pd.read_csv("../datasets/candidates/wmpcand_012523_wmpid.csv")

In [3]:
h.columns

Index(['wmpid', 'genelect_cd', 'CurrCand', 'cand_name', 'cand_id',
       'cand_office', 'cand_office_st', 'cand_office_dist',
       'cand_party_affiliation', 'cand_incumbent_challenger_open_s',
       'dateadded_cd', 'office_wapo', 'result_wapo', 'primarydate_wapo',
       'trumpe_wapo', 'gender_wmp', 'gender_crp', 'latino_wmp', 'latino_crp',
       'race_wmp', 'race_crp1', 'race_crp2', 'race_crpmena', 'hse_cmpt_gen',
       'full_name', 'first_name', 'last_name'],
      dtype='object')

In [4]:
h.genelect_cd.value_counts(dropna=False)

0    2929
1    1366
Name: genelect_cd, dtype: int64

In [5]:
h = h.loc[(h['genelect_cd']==1)&(h['cand_office']=='H')]

In [6]:
h.shape

(1193, 27)

In [7]:
h.head()

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name,first_name,last_name
0,WMPID21,1,Y,"CARL, JERRY LEE, JR",H0AL01055,H,AL,1.0,REP,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Jerry Carl,Jerry,Carl
1,WMPID24,1,Y,"HARVEY-HALL, PHYLLIS",H0AL02202,H,AL,2.0,DEM,CHALLENGER,...,0.0,n,Black,B,NaN,n,0,Phyllis Harvey-Hall,Phyllis,Harvey-Hall
2,WMPID27,1,Y,"SEWELL, TERRI A.",H0AL07086,H,AL,7.0,DEM,INCUMBENT,...,0.0,N,Black,B,NaN,N,0,Terri A Sewell,Terri,Sewell
3,WMPID28,1,Y,"CRAWFORD, ERIC ALAN RICK",H0AR01083,H,AR,1.0,REP,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Rick Crawford,Rick,Crawford
4,WMPID30,1,Y,"WOMACK, STEVE",H0AR03055,H,AR,3.0,REP,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Steve Womack,Steve,Womack


In [8]:
h = h[['wmpid', 'cand_name', 'full_name', 'cand_office_st', 'cand_party_affiliation']]

In [9]:
h.head()

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation
0,WMPID21,"CARL, JERRY LEE, JR",Jerry Carl,AL,REP
1,WMPID24,"HARVEY-HALL, PHYLLIS",Phyllis Harvey-Hall,AL,DEM
2,WMPID27,"SEWELL, TERRI A.",Terri A Sewell,AL,DEM
3,WMPID28,"CRAWFORD, ERIC ALAN RICK",Rick Crawford,AR,REP
4,WMPID30,"WOMACK, STEVE",Steve Womack,AR,REP


In [10]:
len(h['wmpid'].unique())

1193

### Load the data cleaned by RA Jasmine

Note: 
- Jasmine's clean-up is based on a previous scraping result which is slightly different from ./data/bp2022_sen_scraped_face.csv
- wmpid was added to this file using both fuzzymatch and human coding

In [11]:
house_c = pd.read_excel("./data/bp2022_house_scraped_face_jasmine.xlsx")

In [12]:
house_c.shape

(1286, 23)

In [13]:
house_c.head(2)

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,...,hc_face_url,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname,hc_office_district,hc_office_district_note
0,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0,REP,Sarah Palin,Sarah Palin,https://s3.amazonaws.com/ballotpedia-api4/file...,Sarah Palin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0,REP,Nicholas Begich,Nicholas Begich,https://s3.amazonaws.com/ballotpedia-api4/file...,Nicholas Begich,...,NaN,Nicholas,Begich,NaN,III,NaN,NaN,NaN,NaN,NaN


Use Jasmine's urls when scraped bp_url is missing

In [14]:
house_c['bp_face_url']= house_c['bp_url']

In [15]:
house_c['bp_face_url']=house_c.apply(
    lambda row: row['hc_face_url'] if pd.isnull(row['bp_face_url']) else row['bp_face_url'],
    axis=1)

In [16]:
house_c.head(2)

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,...,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname,hc_office_district,hc_office_district_note,bp_face_url
0,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0,REP,Sarah Palin,Sarah Palin,https://s3.amazonaws.com/ballotpedia-api4/file...,Sarah Palin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...
1,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0,REP,Nicholas Begich,Nicholas Begich,https://s3.amazonaws.com/ballotpedia-api4/file...,Nicholas Begich,...,Nicholas,Begich,NaN,III,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...


In [17]:
len(house_c.loc[house_c['bp_url'].isna()])

382

In [18]:
house_c.loc[house_c['bp_face_url'].isna()]

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,...,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname,hc_office_district,hc_office_district_note,bp_face_url
24,WMPID4534,"SMITH, JONATHAN",Jonathan Smith,AL,2,LIB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,WMPID1702,"MOSLEY, MARK",Mark Mosley,GA,8,LIB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,WMPID3167,"SINGLETON, NEIL",Neil Singleton,GA,12,REP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
532,WMPID3163,"BADGER, JOHN ROBERT",John Robert Badger,LA,5,IND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,WMPID301,"HALL, JULIE",Julie Hall,MA,4,REP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
710,WMPID2752,"ZYGMOND, CURT ANDREW MR. SR.",Curt Andrew Sr. Zygmond,MT,2,IND,NaN,NaN,NaN,NaN,...,Curt,Zygmond,Andrew,Sr.,NaN,NaN,NaN,NaN,NaN,NaN
720,WMPID2175,"WEYER, DANIEL FRED",Daniel Fred Weyer,NE,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,WMPID3909,"PARK, CHARLES",Charles Park,NJ,1,IND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
903,WMPID4798,"REGENOR, JAMES ALLEN COL",James Allen Regenor,NY,23,W,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
959,WMPID2045,"GORMAN, LYNN CAROL",Lynn Carol Gorman,OH,13,IND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
url=house_c[['wmpid', 'bp_face_url']]

### Finalize face urls

In [20]:
df = h.merge(url, on='wmpid', how='left', indicator=True)

In [21]:
df._merge.value_counts()

both          1181
left_only       12
right_only       0
Name: _merge, dtype: int64

Double check the missing ones from Jasmine's file

In [22]:
df.loc[df._merge=='left_only']

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,bp_face_url,_merge
31,WMPID115,"NORTON, ELEANOR HOLMES",Eleanor Holmes Norton,DC,DEM,NaN,left_only
185,WMPID4631,"BYE, CHRIS",Chris Bye,AK,LIB,NaN,left_only
290,WMPID2605,"LONG, TIMOTHY LOUIS",Timothy Louis Long,CO,UN,NaN,left_only
298,WMPID2101,"RIMENSNYDER, NELSON F",Nelson Rimensnyder,DC,REP,NaN,left_only
299,WMPID4991,"MAJORS, BRUCE POWELL MR.",Bruce Powell Majors,DC,LIB,NaN,left_only
423,WMPID3894,"NELSON, TELENA C.",Telena Nelson,GU,DEM,NaN,left_only
424,WMPID4351,"PAT, JUDITH T WON",Judith T Won Pat,GU,DEM,NaN,left_only
858,WMPID858,"PLASKETT, STACEY",Stacey Plaskett,VI,DEM,NaN,left_only
892,WMPID865,"AMATA, AUMUA",Amata Coleman Radewagen,AS,REP,NaN,left_only
1062,WMPID1083,"STRACUZZI, NATALE N.",Natale Stracuzzi,DC,DCG,NaN,left_only


In [23]:
df.loc[df['wmpid']=='WMPID4631', 'bp_face_url']='https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Chris-Bye.PNG'
df.loc[df['wmpid']=='WMPID2605', 'bp_face_url']='https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/TimLong.jpg'

In [24]:
df.loc[df._merge=='left_only']

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,bp_face_url,_merge
31,WMPID115,"NORTON, ELEANOR HOLMES",Eleanor Holmes Norton,DC,DEM,NaN,left_only
185,WMPID4631,"BYE, CHRIS",Chris Bye,AK,LIB,https://s3.amazonaws.com/ballotpedia-api4/file...,left_only
290,WMPID2605,"LONG, TIMOTHY LOUIS",Timothy Louis Long,CO,UN,https://s3.amazonaws.com/ballotpedia-api4/file...,left_only
298,WMPID2101,"RIMENSNYDER, NELSON F",Nelson Rimensnyder,DC,REP,NaN,left_only
299,WMPID4991,"MAJORS, BRUCE POWELL MR.",Bruce Powell Majors,DC,LIB,NaN,left_only
423,WMPID3894,"NELSON, TELENA C.",Telena Nelson,GU,DEM,NaN,left_only
424,WMPID4351,"PAT, JUDITH T WON",Judith T Won Pat,GU,DEM,NaN,left_only
858,WMPID858,"PLASKETT, STACEY",Stacey Plaskett,VI,DEM,NaN,left_only
892,WMPID865,"AMATA, AUMUA",Amata Coleman Radewagen,AS,REP,NaN,left_only
1062,WMPID1083,"STRACUZZI, NATALE N.",Natale Stracuzzi,DC,DCG,NaN,left_only


Check all house candidates without face urls

In [25]:
df.loc[df.bp_face_url.isna()]

,wmpid,cand_name,full_name,cand_office_st,cand_party_affiliation,bp_face_url,_merge
31,WMPID115,"NORTON, ELEANOR HOLMES",Eleanor Holmes Norton,DC,DEM,NaN,left_only
83,WMPID3163,"BADGER, JOHN ROBERT",John Robert Badger,LA,IND,NaN,both
85,WMPID301,"HALL, JULIE",Julie Hall,MA,REP,NaN,both
181,WMPID2813,"BURKS, COURTNEY YVONNE",Courtney Yvonne Burks,SC,IND,NaN,both
186,WMPID4534,"SMITH, JONATHAN",Jonathan Smith,AL,LIB,NaN,both
298,WMPID2101,"RIMENSNYDER, NELSON F",Nelson Rimensnyder,DC,REP,NaN,left_only
299,WMPID4991,"MAJORS, BRUCE POWELL MR.",Bruce Powell Majors,DC,LIB,NaN,left_only
412,WMPID1702,"MOSLEY, MARK",Mark Mosley,GA,LIB,NaN,both
419,WMPID3167,"SINGLETON, NEIL",Neil Singleton,GA,REP,NaN,both
423,WMPID3894,"NELSON, TELENA C.",Telena Nelson,GU,DEM,NaN,left_only


In [26]:
df['face_2022']='yes'

In [27]:
df['face_category']='house_general'

In [28]:
df = df[['wmpid', 'bp_face_url', 'face_2022', 'face_category']]

In [29]:
df=df.rename(columns={'bp_face_url':'face_url'})

In [30]:
df.shape

(1193, 4)

In [31]:
df.head()

,wmpid,face_url,face_2022,face_category
0,WMPID21,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,house_general
1,WMPID24,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,house_general
2,WMPID27,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,house_general
3,WMPID28,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,house_general
4,WMPID30,https://s3.amazonaws.com/ballotpedia-api4/file...,yes,house_general


In [32]:
df.to_csv("./data/face_url_2022_house_general.csv", index=True)